# Base Processor Workflow

This notebook shows what is necessary to use the flow with data and what the processor does hand how, to deal with this.

It shows the low level base Workflow, that is also used by the API in the background.

In [1]:
import flowcode
import processing
import res_flow_vis as visual
import externalize as ext

import torch
import numpy as np

In [2]:
#Filename for a given run
filename = "leavout_MttZ_all_CL2_24_10_512_8_lo[66, 20, 88, 48, 5]" #"leavout_MttZ_MWs_CL2_24_10_512_8_lo5"

In [2]:
#Initiate the processor
mpc = processing.Processor_cond()

The processor is an object that keeps track of the transformations applied to the data when normalizing for training, remembers the component names of the data before detaching to torch tensors and reverses all this when making inference.

#### 1. Load data

In [3]:
#Load raw data, this loads the data from the files and processes it to the correct format
Galaxies_raw = mpc.get_data("all_sims")

In [4]:
#Note it also registers used quantitiy names
mpc.component_names

{'stars': ['x', 'y', 'z', 'vx', 'vy', 'vz', 'Z', 'feh', 'ofe', 'mass', 'age']}

In [10]:
#Data format, see the documentation of Processor_cond for more details
example_galaxy = Galaxies_raw[0]

for key, value in example_galaxy.items():
    print(f'key "{key}" contains a {type(value).__name__}:')
    display(value)

key "stars" contains a DataFrame:


,x,y,z,vx,vy,vz,Z,feh,ofe,mass,age
0,8.320762,13.947057,0.648117,-15.820096,15.730662,25.561372,0.000000,-41.507547,0.466361,7830.175642,13.522674
1,1.402683,-1.459748,1.174516,20.920114,-30.466823,-15.565336,0.000000,-41.507547,0.466361,7830.176248,13.520988
2,-0.354377,4.079226,1.949618,1.926594,-109.700875,11.080549,0.000000,-41.507547,0.466361,7830.176248,13.520145
3,0.715224,-3.148012,-2.108494,35.815449,13.556570,-18.749363,0.000000,-41.507547,0.466361,7830.176248,13.519302
4,-5.521244,7.460683,-2.849450,-12.839982,-9.406608,-12.336868,0.000000,-41.507547,0.466361,7830.176248,13.519302
...,...,...,...,...,...,...,...,...,...,...,...
70473,4.176173,-7.198771,-1.684753,27.148138,46.779784,19.308997,0.002247,-1.014137,0.063585,13192.044900,0.000855
70474,4.032141,-7.345427,-1.628008,11.584631,20.404142,14.985289,0.002307,-1.002988,0.064091,13193.044887,0.000012
70475,3.987357,-7.469818,-1.605336,10.797821,16.979302,12.951498,0.002271,-1.010247,0.064449,13193.044887,0.000012
70476,4.009991,-7.326869,-1.702954,9.155321,25.086696,13.437631,0.002278,-1.005108,0.060350,13193.044887,0.000012


key "galaxy" contains a dict:


{'M_dm': 109636618947.82254,
 'M_stars': 576316871.6737778,
 'N_stars': 70478,
 'id': 0,
 'NIHAO_id': 'g1.05e11'}

In [28]:
#There is a nice function to get say the array of the stellar masses
M_stars_raw = mpc.get_array(Galaxies_raw, "galaxy", "M_stars")
print(len(M_stars_raw))
M_stars_raw[:5]

95


array([5.76316872e+08, 3.34649413e+06, 8.58793373e+08, 6.64246619e+06,
       2.02812230e+09])

#### 2. Clean data

In [5]:
#Clean data with specific algorithm
#This algorithm will probably be needed to be changed for different datasets, see below
Galaxies_cleaned = mpc.constraindata(Galaxies_raw, N_min=500)

Cut out 5 of 95 galaxies, 2072015 of 34878379 stars (~6%).


In [11]:
#This will adjust e.g. total stellar mass
id_0_galaxy = list(filter(lambda x: x["galaxy"]["id"] == 0, Galaxies_cleaned))[0]

id_0_galaxy["galaxy"]

{'M_dm': 109636618947.82254,
 'M_stars': 547990391.3124597,
 'N_stars': 66883,
 'id': 0,
 'NIHAO_id': 'g1.05e11'}

#### 3. Choose subset

In [12]:
#Chose a subset of the data
#1. Define conditions to be used
#Done by supplying a function, that computes conditions from a galaxy (dict as above) and returns them with condition names (dict or DataFrame)
cond_fn = ext.cond_M_stars_2age_avZ

#2. Name the components to be used
#Here ignore a stars mass as they are all equal due to simulation restrictions
comp_use = ["x", "y", "z", "vx", "vy", "vz", "Z", "feh", "ofe", "age"]

#3. Define the subset to be used (MWs, all, etc.)

#Done by supplying a function that takes a galaxy and returns a boolean if it should be used
#Subset to view for comaprison (e.g. all galaxies, no leavout i.e. contains validation set):
#This function will leavout all galaxies that have galaxy["galaxy"]["id"] in the supplied list
use_fn_view = ext.construct_all_galaxies_leavout("id", [])
Galaxies = mpc.choose_subset(Galaxies_cleaned, use_fn = use_fn_view, cond_fn=cond_fn, comp_use=comp_use)

#Subset to train on (e.g. all galaxies, leavout 5 as validation set):
leavout_idices = [66, 20, 88, 48, 5]
use_fn_train = ext.construct_all_galaxies_leavout("id", leavout_idices)
Galaxies_train = mpc.choose_subset(Galaxies_cleaned, use_fn = use_fn_train, cond_fn=cond_fn, comp_use=comp_use)

Chose 90 of 90 galaxies.
Chose 85 of 90 galaxies.


In [14]:
#This will add the conditions to the data under the key "parameters"
id_0_galaxy = list(filter(lambda x: x["galaxy"]["id"] == 0, Galaxies))[0]

id_0_galaxy["parameters"]

,M_stars,tau50,tau10,Z_av
0,5.479904e+08,4.942476,1.217085,0.001316


In [19]:
#Note that the conditions are now also registered
print(mpc.cond_names)

#And that the galaxies with id in leavout_idices are not in the training data
leavout_galaxies = filter(lambda x: x["galaxy"]["id"] in leavout_idices, Galaxies_train)
print(len(list(leavout_galaxies)))

#But are in the view data
leavout_galaxies = filter(lambda x: x["galaxy"]["id"] in leavout_idices, Galaxies)
print(len(list(leavout_galaxies)))

{'stars': [], 'galaxy': ['M_stars', 'tau50', 'tau10', 'Z_av']}
0
5


### Training

In [7]:
#Choose device
device = "cpu"

In [8]:
#Hyperparameters of the flow
LAYER_TYPE = flowcode.NSF_CL2
N_LAYERS = 14
COND_NAMES = mpc.cond_names["galaxy"]
DIM_COND = len(COND_NAMES)
DIM_NOTCOND = len(Galaxies_train[0]["stars"]) - DIM_COND
SPLIT = 0.5
K = 10
B = 3
BASE_NETWORK = flowcode.MLP
BASE_NETWORK_N_LAYERS = 4
BASE_NETWORK_N_HIDDEN = 128
BASE_NETWORK_LEAKY_RELU_SLOPE = 0.2

SPLIT = {"split":SPLIT} if LAYER_TYPE == flowcode.NSF_CL else {}

In [9]:
#Instantiate the model
model = flowcode.NSFlow(N_LAYERS, DIM_NOTCOND, DIM_COND, LAYER_TYPE, **SPLIT, K=K, B=B, network=BASE_NETWORK, network_args=(BASE_NETWORK_N_HIDDEN,BASE_NETWORK_N_LAYERS,BASE_NETWORK_LEAKY_RELU_SLOPE))
model = model.to(device)
#Load pre-trained model
#model.load_state_dict(torch.load("saves/leavout_M_star_MWs_CL2_24_10_512_8_lo1.pth"))

In [10]:
#Training hyperparameters
N_EPOCHS = 12
INIT_LR = 0.00009
GAMMA = 0.998
BATCH_SIZE = 1024

In [11]:
#The flow should be trained in normalized coordinates
#Also we want e.g. total stellar mass to be learned in log

#M_stars should be learned in log
LOG_LEARN = ["M_stars"]

#Define the transformations to be used
transformations = (np.log10, )
#Define the manes affected by the transformations (i.e. components of trf_names[i] are supplied to transformations[i])
trf_names = (LOG_LEARN, )
#Define the inverse transformations (these are applied to the model output)
transformations_inv = (lambda x: 10**x, )
#Define the logdets of the transformations needed if the pdf is to be computed
logdets = (ext.logdet_log10,)

#Now diststack will format the data to the correct format for the flow (one array of iid samples)
#Essentially, it will stack all stars of all galaxies together and apend the conditions of the corresponding to each star
#Data_to_flow will then normalize the data and apply the transformations
#The returned data is to be used for training directly
#IMPORTANT: Do not modify the returned data in any way:
#The processor remembers e.g. the order of the components and will assume those when obtaining a sample from the flow and renaming them
#Also remember e.g. (f°g)^-1 = g^-1 ° f^-1
#so if you apply a transformation to the data, you would need to apply the inverse transformation to the flow output before the processor can use it
#which is not possible as the processor directly applies the inverse transformation to the flow output
Data_flow = mpc.Data_to_flow(mpc.diststack(Galaxies_train), transformations, trf_names, transformations_inv, transformation_logdets=logdets)

In [ ]:
#Note that you can reproduce this exact normalization on other data by using mpc.reproduce_normalization
#This can be used to transform some other data (e.g. your desired conditions when sampling from the flow) to normalized coordinates
#the flow was trained on (I.e. it will not recalculate the normalization but use the one it was trained on)

#The inverse is achieved with sample_to_Data

In [ ]:
#Training takes long, so usually we do not want it in a notebook but in a nohup background process
#E.g. it is possible to use the cond_trainer.py script to train the model
#this will export the model and data accordingly to the cond_trainer folder
torch.save(Data_flow, "cond_trainer/data_cond_trainer.pth")
torch.save(model, "cond_trainer/model_cond_trainer.pth")
np.save("cond_trainer/params_cond_trainer.npy", np.append(COND_NAMES,np.array([N_EPOCHS,INIT_LR,BATCH_SIZE,GAMMA])))
np.save("cond_trainer/filename_cond_trainer.npy", filename)

In [ ]:
#Start background training
#nohup python cond_trainer.py <name_suffix> <optional:GPU id> &
#Will save model in saves folder with loss history and train time (last entry in loss history)
#Will flush training information to a textfile containing the name suffix

In [ ]:
#...OR train directly in notebook/current process
import time
train_loss_saver = []
start = time.perf_counter()
flowcode.train_flow(model, Data_flow, COND_NAMES, N_EPOCHS, lr=INIT_LR, batch_size=BATCH_SIZE, loss_saver=train_loss_saver, gamma=GAMMA)
end = time.perf_counter()
torch.save(model.state_dict(), f"saves/{filename}.pth")
np.save(f"saves/loss_{filename}.npy",np.array(train_loss_saver+[end-start]))

In [12]:
#Load in training results:
model.load_state_dict(torch.load(f"saves/{filename}.pth", map_location=device))
loss_results = np.load(f"saves/loss_{filename}.npy")
loss_results, tot_time = loss_results[:-1], loss_results[-1]/60

### Inference

In [13]:
#Get a sample from the flow
use_GPUs = [9]
import time
start = time.perf_counter()
#Set a condition for the sample
#Here, sample back all galaxies
#I.e. Condition = all galaxy parameters as often as there are stars in the galaxy in a known order
condition = mpc.diststack(Galaxies)[COND_NAMES]

#Get sample with sample_Conditional
#This will automatically:
#1. Normalize the condition (using reproduce_normalization)
#2. Sample from the flow

#Denormalize the sample with sample_to_Data
flow_sample = mpc.sample_to_Data(mpc.sample_Conditional(model, condition, split_size=int(6e5), GPUs=use_GPUs))

#To revert to a galaxy interpretation we need to specify the number of stars belonging in each galaxy
#Again, we use the same as in the data
N_stars_galaxies = mpc.get_array(Galaxies, "galaxy" ,"N_stars")
#or
N_stars_galaxies = np.array([len(galaxy["stars"]) for galaxy in Galaxies])

#Convert sample to galaxy interpretation
flow_sample = mpc.galaxysplit(flow_sample, N_stars_galaxies)

#However this is now list of pandas dataframes, not a list of dictionaries as the original data
#we can convert it back to a list of dictionaries
flow_sample = [{"stars":galaxy_stars} for galaxy_stars in flow_sample]
#In our special case, we can also reinsert the galaxy information that we know from the original data
#This is of course not possible in general
for galaxy_flow, galaxy in zip(flow_sample, Galaxies):
    galaxy_flow["galaxy"] = galaxy["galaxy"]

#Similarly, one can also reinsert the conditions
for galaxy_flow, galaxy in zip(flow_sample, Galaxies):
    galaxy_flow["parameters"] = galaxy["parameters"]

#Format in minutes and seconds
print(f"Time to sample: {int((time.perf_counter()-start)/60)} minutes and {int((time.perf_counter()-start)%60)} seconds")

Time to sample: 26 minutes and 41 seconds


In [ ]:
### Visualize data

In [1]:
#Get multiple galaxy plot
visual.plot_conditional_2(Galaxies, flow_sample, type="N", label=filename, N_unit="massperkpc", color_pass="first", global_grid=True)

NameError: name 'visual' is not defined

In [ ]:
#Get comparison plot of single galaxy

visual.get_result_plots(Galaxies[5], flow_sample[5], label=filename, format_="pdf")

In [ ]:
visual.plot_conditional_histograms(flow_sample, label = filename, log=True)

In [ ]:
visual.loss_plot(loss_results, tot_time=tot_time, savefig=filename)

In [ ]:
#Also the pdf can be computed
#This automatically:
#1. Normalizes the points
#2. Computes the pdf
#3. Denormalizes the pdf i.e. respects the transformation_logdets
Points = mpc.diststack(Galaxies)
log_prob = mpc.log_prob(model, Points, GPUs=use_GPUs, split_size=int(6e5))